In [1]:
import sys
sys.path.append('../..')

import json
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback

from src.Evaluation import plot_model_history
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder, get_Resnet
from src.LoadData import get_all_datasets_test_train_np_arrays
from src.Helpers import append_to_csv

In [2]:
path_to_datasets = "../../datasets"

In [3]:
datasets_test_train_data = get_all_datasets_test_train_np_arrays(path_to_datasets)

In [4]:
models_getter = [get_Time_CNN()]
models_names = ["Time-CNN"]

In [8]:
# tf.debugging.set_log_device_placement(True)
devices = tf.config.list_logical_devices('GPU') # Use this to run training just on GPUs
# devices = tf.config.list_logical_devices()
strategy = tf.distribute.MirroredStrategy(devices)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [10]:
devices

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [5]:
path_persist_results = "./training_res.csv"
if not (csv_path := Path(path_persist_results)).exists():
    csv_path.touch()
if len(csv_path.read_text()) == 0:
    # Only write if the file is empty
    columns = ["dataset_name", "model_name", "test_loss", "test_acc", "confusion_matrix", "history"]
    append_to_csv(path_persist_results, columns)

In [6]:
epochs = 30
batch_size = 10
validation_split = 0.2

In [7]:
# with strategy.scope():
for ds_name, ds_data in tqdm(datasets_test_train_data.items(), unit='dataset'):
    print("Dataset name: ", ds_name)
    x_test, y_test = ds_data["test_data"]
    x_train, y_train = ds_data["train_data"]

    input_size = x_train.shape[1]
    output_size = len(np.unique(y_train))

    for get_model, model_name in tqdm(list(zip(models_getter, models_names)), unit='model', desc=f'Train on "{ds_name}"'):
        print("Model name: ", model_name)
        model = get_model(input_size, output_size)
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[TqdmCallback(verbose=0, desc=model_name)], verbose=0)
        test_loss, test_acc = model.evaluate(x_test, y_test)
        # TODO: add confusion matrix

        row = [ds_name,
              model_name,
              test_loss,
              test_acc,
              None, # TODO: persist confusion matrix to further analysis
              json.dumps(history.history)]
        append_to_csv(path_persist_results, row)

  0%|          | 0/33 [00:00<?, ?dataset/s]

Dataset name:  abnormal_heartbeat


Train on "abnormal_heartbeat":   0%|          | 0/1 [00:00<?, ?model/s]

Model name:  MCDCNN


MCDCNN: 0epoch [00:00, ?epoch/s]

KeyboardInterrupt: 

# Example load training results and display evaluation results

In [ ]:
df_training_res = pd.read_csv(path_persist_results)

In [ ]:
df_training_res

In [ ]:
history = df_training_res['history'][0]

In [ ]:
plot_model_history(json.loads(history), epochs=epochs)